In [44]:
from sklearn import decomposition
import sys, os
import pandas as pd
import numpy as np
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from helper import *
import glob
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Loading losses

In [19]:
all_files = glob.glob(cfg["models"]["autoencoder"]+"*")


In [33]:
all_model_names = set([x.split("/")[-1].replace(".npy","").replace("_test_loss","").replace("_train_loss","") for x in all_files])

In [15]:
models = {path.split("/")[-1] :  keras.models.load_model(path)  for path in model_paths[:2]}

In [54]:

mse = []
model_names = []
loss_type = []

for model_name in all_model_names:
    print(model_name)
    train_e = np.load(cfg["models"]["autoencoder"]+f"{model_name}_test_loss.npy")[-1]
    mse.append(test_e)
    loss_type.append("test")
    test_e  = np.load(cfg["models"]["autoencoder"]+f"{model_name}_train_loss.npy")[-1]
    mse.append(train_e)
    loss_type.append("train")
    model_names.append(model_name)
    model_names.append(model_name)

fully_connected_3L


# PCA errors

In [50]:
X_train = np.load(cfg['files']["raw_train_features_liquid"])
X_test  = np.load(cfg['files']["raw_test_features_liquid"])
print(f"shapes : X_train={X_train.shape},X_test={X_test.shape}")
N_train = len(X_train)
N_test  = len(X_test)
PCA_X_train = X_train.reshape(N_train,-1)
PCA_X_test = X_test.reshape(N_test,-1)
pca = decomposition.PCA(n_components=100)
pca.fit(PCA_X_train)

def mean_squared_error(x,y):
    return np.sum((x-y)**2)/x.size
pca_train_mse = mean_squared_error(PCA_X_train, pca.inverse_transform(pca.transform(PCA_X_train)))
pca_test_mse =  mean_squared_error(PCA_X_test, pca.inverse_transform(pca.transform(PCA_X_test))) 


shapes : X_train=(5292, 3, 600, 2),X_test=(1323, 3, 600, 2)


In [55]:
mse.append(pca_test_mse)
loss_type.append("test")
mse.append(pca_train_mse)
loss_type.append("train")
model_names.append("PCA")
model_names.append("PCA")

# All results

In [56]:
res = pd.DataFrame({"model name":model_names , "mse" :mse , "loss_type": loss_type})
res

,model name,mse,loss_type
0,fully_connected_3L,0.057495,test
1,fully_connected_3L,0.086256,train
2,PCA,0.082692,test
3,PCA,0.079865,train


In [57]:

fig = px.bar(res, x='model name', y='mse',color="loss_type")
fig.write_html(f"{cfg['fig_dir']['embedding']}losses_comparison.html",full_html=False, include_plotlyjs=False)

fig.show()